# Task 3: A/B Hypothesis Testing

## Objective
Statistically validate or reject key hypotheses about risk drivers, which will form the basis of our new segmentation strategy.

## Null Hypotheses to Test:

1. **H₀**: There are no risk differences across provinces
2. **H₀**: There are no risk differences between zip codes
3. **H₀**: There is no significant margin (profit) difference between zip codes
4. **H₀**: There is no significant risk difference between Women and Men

## Metrics:
- **Claim Frequency**: Proportion of policies with at least one claim
- **Claim Severity**: Average claim amount given a claim occurred
- **Margin**: TotalPremium - TotalClaims

## Statistical Tests:
- **Chi-squared test**: For categorical comparisons (claim frequency)
- **T-test / Mann-Whitney U test**: For continuous comparisons (claim severity, margin)
- **ANOVA / Kruskal-Wallis test**: For multiple group comparisons


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
import sys
sys.path.append('../src')
from data_processing import load_and_validate_data
from hypothesis_testing import (
    test_province_risk_differences,
    test_zipcode_risk_differences,
    test_zipcode_margin_differences,
    test_gender_risk_differences,
    test_claim_frequency_difference,
    test_claim_severity_difference,
    prepare_groups
)
from utils import calculate_claim_frequency, calculate_claim_severity

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")


## 1. Data Loading


In [ ]:
# Load the data
data_path = '../data/raw/insurance_data.csv'  # Update this path

try:
    df = load_and_validate_data(data_path)
    print(f"Data loaded successfully!")
    print(f"Shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
except FileNotFoundError:
    print(f"Data file not found at {data_path}")
    print("Please update the data_path variable with the correct path to your insurance data.")
    print("\nCreating sample data for demonstration...")
    # Create sample data
    np.random.seed(42)
    n_samples = 5000
    df = pd.DataFrame({
        'PolicyID': range(n_samples),
        'Province': np.random.choice(['Gauteng', 'Western Cape', 'KwaZulu-Natal', 'Eastern Cape'], n_samples),
        'PostalCode': np.random.choice([1000, 2000, 3000, 4000, 5000], n_samples),
        'Gender': np.random.choice(['Male', 'Female'], n_samples),
        'TotalPremium': np.random.uniform(5000, 50000, n_samples),
        'TotalClaims': np.random.exponential(10000, n_samples) * (np.random.random(n_samples) < 0.3)
    })
    print("Sample dataframe created for demonstration purposes.")


In [ ]:
# Display basic statistics
print("Data Overview:")
print(f"Total policies: {len(df):,}")
if 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    print(f"Overall claim frequency: {calculate_claim_frequency(df):.4f}")
    print(f"Overall claim severity: ZAR {calculate_claim_severity(df):,.2f}")
    print(f"Total Premium: ZAR {df['TotalPremium'].sum():,.2f}")
    print(f"Total Claims: ZAR {df['TotalClaims'].sum():,.2f}")
    print(f"Total Margin: ZAR {(df['TotalPremium'].sum() - df['TotalClaims'].sum()):,.2f}")


## 2. Hypothesis Test 1: Risk Differences Across Provinces

**H₀**: There are no risk differences across provinces


In [ ]:
# Test province risk differences
if 'Province' in df.columns:
    province_results = test_province_risk_differences(df, alpha=0.05)
    
    print("="*80)
    print("HYPOTHESIS TEST 1: PROVINCE RISK DIFFERENCES")
    print("="*80)
    print(f"\nNull Hypothesis: {province_results.get('null_hypothesis', 'N/A')}")
    print(f"\nProvinces tested: {province_results.get('provinces_tested', 'N/A')}")
    print(f"Significance level (α): {province_results.get('alpha', 0.05)}")
    
    # Frequency test results
    freq_test = province_results.get('frequency_test', {})
    print(f"\n--- Claim Frequency Test ---")
    print(f"Test type: {freq_test.get('test_type', 'N/A')}")
    print(f"Statistic: {freq_test.get('statistic', 'N/A'):.4f}")
    print(f"P-value: {freq_test.get('p_value', 'N/A'):.6f}")
    print(f"Conclusion: {freq_test.get('conclusion', 'N/A')}")
    
    # Severity test results
    sev_test = province_results.get('severity_test', {})
    print(f"\n--- Claim Severity Test ---")
    print(f"Test type: {sev_test.get('test_type', 'N/A')}")
    print(f"Statistic: {sev_test.get('statistic', 'N/A'):.4f}")
    print(f"P-value: {sev_test.get('p_value', 'N/A'):.6f}")
    print(f"Conclusion: {sev_test.get('conclusion', 'N/A')}")
    
    print(f"\n--- Overall Conclusion ---")
    print(f"{province_results.get('overall_conclusion', 'N/A')}")
    
    # Business interpretation
    if province_results.get('overall_conclusion', '').startswith('Reject'):
        print("\n📊 BUSINESS INTERPRETATION:")
        print("We reject the null hypothesis. There ARE significant risk differences across provinces.")
        print("This suggests that regional risk adjustment to premiums may be warranted.")
        print("Action: Analyze province-specific loss ratios and consider premium adjustments.")
    else:
        print("\n📊 BUSINESS INTERPRETATION:")
        print("We fail to reject the null hypothesis. There is no significant evidence of risk differences across provinces.")
        print("Action: Continue monitoring, but no immediate premium adjustments needed based on province.")
else:
    print("Province column not found in dataset.")


In [ ]:
# Visualize province risk differences
if 'Province' in df.columns and 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    province_metrics = df.groupby('Province').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum',
        'PolicyID': 'count'
    }).reset_index()
    province_metrics['LossRatio'] = province_metrics['TotalClaims'] / province_metrics['TotalPremium']
    province_metrics['ClaimFrequency'] = df.groupby('Province').apply(
        lambda x: calculate_claim_frequency(x)
    ).values
    province_metrics = province_metrics.sort_values('LossRatio', ascending=False)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Loss Ratio by Province
    axes[0].barh(province_metrics['Province'], province_metrics['LossRatio'], 
                 edgecolor='black', alpha=0.7)
    axes[0].axvline(x=1.0, color='r', linestyle='--', linewidth=2, label='Break-even')
    axes[0].set_xlabel('Loss Ratio')
    axes[0].set_title('Loss Ratio by Province', fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='x')
    
    # Claim Frequency by Province
    axes[1].barh(province_metrics['Province'], province_metrics['ClaimFrequency'], 
                 edgecolor='black', alpha=0.7, color='orange')
    axes[1].set_xlabel('Claim Frequency')
    axes[1].set_title('Claim Frequency by Province', fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.savefig('../figures/province_risk_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\nProvince Metrics:")
    print(province_metrics[['Province', 'PolicyID', 'LossRatio', 'ClaimFrequency']].to_string(index=False))


In [ ]:
# Test zipcode risk differences
if 'PostalCode' in df.columns:
    zipcode_results = test_zipcode_risk_differences(df, alpha=0.05, min_samples_per_zipcode=30)
    
    print("="*80)
    print("HYPOTHESIS TEST 2: ZIPCODE RISK DIFFERENCES")
    print("="*80)
    print(f"\nNull Hypothesis: {zipcode_results.get('null_hypothesis', 'N/A')}")
    print(f"\nZipcodes tested: {zipcode_results.get('zipcodes_tested', 'N/A')}")
    print(f"Total valid zipcodes: {zipcode_results.get('total_valid_zipcodes', 'N/A')}")
    print(f"Significance level (α): {zipcode_results.get('alpha', 0.05)}")
    
    # Frequency test results
    freq_test = zipcode_results.get('frequency_test', {})
    print(f"\n--- Claim Frequency Test ---")
    print(f"Test type: {freq_test.get('test_type', 'N/A')}")
    print(f"Statistic: {freq_test.get('statistic', 'N/A'):.4f}")
    print(f"P-value: {freq_test.get('p_value', 'N/A'):.6f}")
    print(f"Conclusion: {freq_test.get('conclusion', 'N/A')}")
    
    # Severity test results
    sev_test = zipcode_results.get('severity_test', {})
    if sev_test.get('p_value') is not None:
        print(f"\n--- Claim Severity Test ---")
        print(f"Test type: {sev_test.get('test_type', 'N/A')}")
        print(f"Statistic: {sev_test.get('statistic', 'N/A'):.4f}")
        print(f"P-value: {sev_test.get('p_value', 'N/A'):.6f}")
        print(f"Conclusion: {sev_test.get('conclusion', 'N/A')}")
    
    print(f"\n--- Overall Conclusion ---")
    print(f"{zipcode_results.get('overall_conclusion', 'N/A')}")
    
    # Business interpretation
    if zipcode_results.get('overall_conclusion', '').startswith('Reject'):
        print("\n📊 BUSINESS INTERPRETATION:")
        print("We reject the null hypothesis. There ARE significant risk differences between zip codes.")
        print("This suggests that location-based risk adjustment to premiums may be warranted.")
        print("Action: Analyze zipcode-specific loss ratios and consider granular premium adjustments.")
    else:
        print("\n📊 BUSINESS INTERPRETATION:")
        print("We fail to reject the null hypothesis. No significant evidence of risk differences between zip codes.")
        print("Action: Continue monitoring, but no immediate premium adjustments needed based on zipcode.")
else:
    print("PostalCode column not found in dataset.")


## 4. Hypothesis Test 3: Margin Differences Between Zip Codes

**H₀**: There is no significant margin (profit) difference between zip codes


In [ ]:
# Test zipcode margin differences
if 'PostalCode' in df.columns and 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    margin_results = test_zipcode_margin_differences(df, alpha=0.05, min_samples_per_zipcode=30)
    
    print("="*80)
    print("HYPOTHESIS TEST 3: ZIPCODE MARGIN DIFFERENCES")
    print("="*80)
    print(f"\nNull Hypothesis: {margin_results.get('null_hypothesis', 'N/A')}")
    print(f"\nZipcodes tested: {margin_results.get('zipcodes_tested', 'N/A')}")
    print(f"Total valid zipcodes: {margin_results.get('total_valid_zipcodes', 'N/A')}")
    print(f"Significance level (α): {margin_results.get('alpha', 0.05)}")
    
    print(f"\n--- Margin Test ---")
    print(f"Test type: {margin_results.get('test_type', 'N/A')}")
    print(f"Statistic: {margin_results.get('statistic', 'N/A'):.4f}")
    print(f"P-value: {margin_results.get('p_value', 'N/A'):.6f}")
    print(f"Conclusion: {margin_results.get('conclusion', 'N/A')}")
    
    # Business interpretation
    if margin_results.get('reject_null', False):
        print("\n📊 BUSINESS INTERPRETATION:")
        print("We reject the null hypothesis. There ARE significant margin differences between zip codes.")
        print("This suggests that profitability varies significantly by location.")
        print("Action: Identify high-margin zipcodes for targeted marketing and low-margin zipcodes for premium review.")
    else:
        print("\n📊 BUSINESS INTERPRETATION:")
        print("We fail to reject the null hypothesis. No significant evidence of margin differences between zip codes.")
        print("Action: Profitability appears relatively uniform across zipcodes.")
else:
    print("Required columns (PostalCode, TotalPremium, TotalClaims) not found in dataset.")


In [ ]:
# Visualize margin differences by zipcode
if 'PostalCode' in df.columns and 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    zipcode_counts = df['PostalCode'].value_counts()
    valid_zipcodes = zipcode_counts[zipcode_counts >= 30].index[:10]  # Top 10
    
    zipcode_margins = []
    for zipcode in valid_zipcodes:
        zipcode_data = df[df['PostalCode'] == zipcode]
        margin = (zipcode_data['TotalPremium'].sum() - zipcode_data['TotalClaims'].sum()) / len(zipcode_data)
        zipcode_margins.append({
            'PostalCode': zipcode,
            'AvgMargin': margin,
            'PolicyCount': len(zipcode_data)
        })
    
    margin_df = pd.DataFrame(zipcode_margins).sort_values('AvgMargin', ascending=False)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.barh(margin_df['PostalCode'].astype(str), margin_df['AvgMargin'], 
           edgecolor='black', alpha=0.7, color='green')
    ax.set_xlabel('Average Margin per Policy (ZAR)', fontweight='bold')
    ax.set_title('Average Margin by Zipcode (Top 10)', fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.savefig('../figures/zipcode_margin_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\nTop 10 Zipcodes by Average Margin:")
    print(margin_df.to_string(index=False))


## 5. Hypothesis Test 4: Risk Differences Between Women and Men

**H₀**: There is no significant risk difference between Women and Men


In [ ]:
# Test gender risk differences
if 'Gender' in df.columns:
    gender_results = test_gender_risk_differences(df, alpha=0.05)
    
    print("="*80)
    print("HYPOTHESIS TEST 4: GENDER RISK DIFFERENCES")
    print("="*80)
    print(f"\nNull Hypothesis: {gender_results.get('null_hypothesis', 'N/A')}")
    print(f"Significance level (α): {gender_results.get('alpha', 0.05)}")
    
    # Frequency test results
    freq_test = gender_results.get('frequency_test', {})
    print(f"\n--- Claim Frequency Test ---")
    print(f"Test type: {freq_test.get('test_name', 'N/A')}")
    print(f"Group A frequency: {freq_test.get('group_a_frequency', 'N/A'):.4f}")
    print(f"Group B frequency: {freq_test.get('group_b_frequency', 'N/A'):.4f}")
    print(f"Difference: {freq_test.get('frequency_difference', 'N/A'):.4f} ({freq_test.get('frequency_difference_pct', 'N/A'):.2f}%)")
    print(f"Chi-squared statistic: {freq_test.get('chi2_statistic', 'N/A'):.4f}")
    print(f"P-value: {freq_test.get('p_value', 'N/A'):.6f}")
    print(f"Conclusion: {freq_test.get('conclusion', 'N/A')}")
    
    # Severity test results
    sev_test = gender_results.get('severity_test', {})
    if 'error' not in sev_test:
        print(f"\n--- Claim Severity Test ---")
        print(f"Test type: {sev_test.get('test_name', 'N/A')}")
        print(f"Group A severity: ZAR {sev_test.get('group_a_severity', 'N/A'):,.2f}")
        print(f"Group B severity: ZAR {sev_test.get('group_b_severity', 'N/A'):,.2f}")
        print(f"Difference: ZAR {sev_test.get('severity_difference', 'N/A'):,.2f} ({sev_test.get('severity_difference_pct', 'N/A'):.2f}%)")
        print(f"Statistic: {sev_test.get('statistic', 'N/A'):.4f}")
        print(f"P-value: {sev_test.get('p_value', 'N/A'):.6f}")
        print(f"Conclusion: {sev_test.get('conclusion', 'N/A')}")
    else:
        print(f"\n--- Claim Severity Test ---")
        print(f"Error: {sev_test.get('error', 'N/A')}")
    
    print(f"\n--- Overall Conclusion ---")
    print(f"{gender_results.get('overall_conclusion', 'N/A')}")
    
    # Business interpretation
    if gender_results.get('overall_conclusion', '').startswith('Reject'):
        print("\n📊 BUSINESS INTERPRETATION:")
        print("We reject the null hypothesis. There ARE significant risk differences between genders.")
        if freq_test.get('frequency_difference', 0) > 0:
            print("One gender shows higher claim frequency, suggesting potential for gender-based premium adjustments.")
        if sev_test.get('severity_difference', 0) != 0:
            print("One gender shows different claim severity, which should be considered in pricing.")
        print("Action: Review gender-based risk profiles and consider if premium adjustments are warranted.")
        print("Note: Ensure compliance with local regulations regarding gender-based pricing.")
    else:
        print("\n📊 BUSINESS INTERPRETATION:")
        print("We fail to reject the null hypothesis. No significant evidence of risk differences between genders.")
        print("Action: Risk profiles appear similar across genders. No immediate premium adjustments needed.")
else:
    print("Gender column not found in dataset.")


In [ ]:
# Visualize gender risk differences
if 'Gender' in df.columns and 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    gender_metrics = df.groupby('Gender').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum',
        'PolicyID': 'count'
    }).reset_index()
    gender_metrics['LossRatio'] = gender_metrics['TotalClaims'] / gender_metrics['TotalPremium']
    gender_metrics['ClaimFrequency'] = df.groupby('Gender').apply(
        lambda x: calculate_claim_frequency(x)
    ).values
    gender_metrics['ClaimSeverity'] = df.groupby('Gender').apply(
        lambda x: calculate_claim_severity(x)
    ).values
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # Loss Ratio
    axes[0].bar(gender_metrics['Gender'], gender_metrics['LossRatio'], 
               edgecolor='black', alpha=0.7, color=['pink', 'lightblue'])
    axes[0].axhline(y=1.0, color='r', linestyle='--', linewidth=2, label='Break-even')
    axes[0].set_ylabel('Loss Ratio')
    axes[0].set_title('Loss Ratio by Gender', fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Claim Frequency
    axes[1].bar(gender_metrics['Gender'], gender_metrics['ClaimFrequency'], 
               edgecolor='black', alpha=0.7, color=['pink', 'lightblue'])
    axes[1].set_ylabel('Claim Frequency')
    axes[1].set_title('Claim Frequency by Gender', fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # Claim Severity
    axes[2].bar(gender_metrics['Gender'], gender_metrics['ClaimSeverity'], 
               edgecolor='black', alpha=0.7, color=['pink', 'lightblue'])
    axes[2].set_ylabel('Claim Severity (ZAR)')
    axes[2].set_title('Claim Severity by Gender', fontweight='bold')
    axes[2].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('../figures/gender_risk_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\nGender Metrics:")
    print(gender_metrics[['Gender', 'PolicyID', 'LossRatio', 'ClaimFrequency', 'ClaimSeverity']].to_string(index=False))


## 6. Summary of All Hypothesis Tests


In [ ]:
# Create summary table of all hypothesis tests
summary_data = []

# Test 1: Provinces
if 'Province' in df.columns:
    province_results = test_province_risk_differences(df, alpha=0.05)
    summary_data.append({
        'Hypothesis': 'H₀: No risk differences across provinces',
        'Test Type': 'Chi-squared (frequency) + Kruskal-Wallis (severity)',
        'P-value (Frequency)': province_results.get('frequency_test', {}).get('p_value', 'N/A'),
        'P-value (Severity)': province_results.get('severity_test', {}).get('p_value', 'N/A'),
        'Conclusion': province_results.get('overall_conclusion', 'N/A')
    })

# Test 2: Zipcodes (Risk)
if 'PostalCode' in df.columns:
    zipcode_results = test_zipcode_risk_differences(df, alpha=0.05, min_samples_per_zipcode=30)
    summary_data.append({
        'Hypothesis': 'H₀: No risk differences between zip codes',
        'Test Type': 'Chi-squared (frequency) + Kruskal-Wallis (severity)',
        'P-value (Frequency)': zipcode_results.get('frequency_test', {}).get('p_value', 'N/A'),
        'P-value (Severity)': zipcode_results.get('severity_test', {}).get('p_value', 'N/A'),
        'Conclusion': zipcode_results.get('overall_conclusion', 'N/A')
    })

# Test 3: Zipcodes (Margin)
if 'PostalCode' in df.columns:
    margin_results = test_zipcode_margin_differences(df, alpha=0.05, min_samples_per_zipcode=30)
    summary_data.append({
        'Hypothesis': 'H₀: No margin difference between zip codes',
        'Test Type': 'Kruskal-Wallis',
        'P-value (Frequency)': 'N/A',
        'P-value (Severity)': margin_results.get('p_value', 'N/A'),
        'Conclusion': margin_results.get('conclusion', 'N/A')
    })

# Test 4: Gender
if 'Gender' in df.columns:
    gender_results = test_gender_risk_differences(df, alpha=0.05)
    summary_data.append({
        'Hypothesis': 'H₀: No risk difference between Women and Men',
        'Test Type': 'Chi-squared (frequency) + T-test/Mann-Whitney (severity)',
        'P-value (Frequency)': gender_results.get('frequency_test', {}).get('p_value', 'N/A'),
        'P-value (Severity)': gender_results.get('severity_test', {}).get('p_value', 'N/A'),
        'Conclusion': gender_results.get('overall_conclusion', 'N/A')
    })

summary_df = pd.DataFrame(summary_data)
print("="*100)
print("HYPOTHESIS TESTING SUMMARY")
print("="*100)
print(summary_df.to_string(index=False))
print("="*100)


In [ ]:
# Generate business recommendations
print("="*100)
print("BUSINESS RECOMMENDATIONS")
print("="*100)

recommendations = []

# Province recommendations
if 'Province' in df.columns:
    province_results = test_province_risk_differences(df, alpha=0.05)
    if province_results.get('overall_conclusion', '').startswith('Reject'):
        recommendations.append({
            'Area': 'Province-Based Pricing',
            'Finding': 'Significant risk differences across provinces detected',
            'Recommendation': 'Implement province-specific premium adjustments. Analyze loss ratios by province and adjust premiums accordingly.',
            'Priority': 'High'
        })
    else:
        recommendations.append({
            'Area': 'Province-Based Pricing',
            'Finding': 'No significant risk differences across provinces',
            'Recommendation': 'Continue monitoring. No immediate premium adjustments needed based on province alone.',
            'Priority': 'Low'
        })

# Zipcode recommendations
if 'PostalCode' in df.columns:
    zipcode_results = test_zipcode_risk_differences(df, alpha=0.05, min_samples_per_zipcode=30)
    margin_results = test_zipcode_margin_differences(df, alpha=0.05, min_samples_per_zipcode=30)
    
    if zipcode_results.get('overall_conclusion', '').startswith('Reject'):
        recommendations.append({
            'Area': 'Zipcode-Based Pricing',
            'Finding': 'Significant risk differences between zipcodes detected',
            'Recommendation': 'Consider granular location-based pricing. Identify high-risk zipcodes for premium increases and low-risk zipcodes for competitive pricing.',
            'Priority': 'High'
        })
    
    if margin_results.get('reject_null', False):
        recommendations.append({
            'Area': 'Zipcode Profitability',
            'Finding': 'Significant margin differences between zipcodes',
            'Recommendation': 'Target high-margin zipcodes for marketing expansion. Review low-margin zipcodes for premium optimization.',
            'Priority': 'Medium'
        })

# Gender recommendations
if 'Gender' in df.columns:
    gender_results = test_gender_risk_differences(df, alpha=0.05)
    if gender_results.get('overall_conclusion', '').startswith('Reject'):
        recommendations.append({
            'Area': 'Gender-Based Pricing',
            'Finding': 'Significant risk differences between genders detected',
            'Recommendation': 'Review gender-based risk profiles. Ensure compliance with local regulations before implementing gender-based pricing adjustments.',
            'Priority': 'Medium (with regulatory review)'
        })
    else:
        recommendations.append({
            'Area': 'Gender-Based Pricing',
            'Finding': 'No significant risk differences between genders',
            'Recommendation': 'Risk profiles are similar across genders. No gender-based premium adjustments needed.',
            'Priority': 'Low'
        })

rec_df = pd.DataFrame(recommendations)
print("\n")
print(rec_df.to_string(index=False))
print("\n" + "="*100)
print("END OF HYPOTHESIS TESTING ANALYSIS")
print("="*100)
